### Create a model using nn.Linear 

In [125]:
import torch
from torch import nn
import matplotlib.pyplot as plt

#### Creating dataset and spliting

In [126]:
weight = 0.7
bias = 0.3

X = torch.arange(start=1, end=100, step=0.3).unsqueeze(dim=1)
y = weight * X + bias

# Spliting data to train and test
train_split = int(0.7 * len(X))
x_train, y_train = X[:train_split], y[:train_split]
x_test, y_test = X[train_split:], y[train_split:]

### Model Creation

In [127]:
# Create a linear model by subclassing nn.Module
class LinearLayerRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Use nn.Linear() for creating the model parameters
        self.linearLayer = nn.Linear(in_features=1, out_features=1)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return self.linearLayer(X)

# Set the manual seed
torch.manual_seed(42)

# Creating the instance of the model
linearModel = LinearLayerRegressionModel()

linearModel,linearModel.state_dict()

(LinearLayerRegressionModel(
   (linearLayer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linearLayer.weight', tensor([[0.7645]])),
              ('linearLayer.bias', tensor([0.8300]))]))

### Using "GPU" for model if available

In [128]:
# Set the device to cuda if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [129]:
# Check the model's current device
next(linearModel.parameters()).device

device(type='cpu')

In [130]:
# Set the model to the device
linearModel.to(device)

next(linearModel.parameters()).device

device(type='cuda', index=0)

## Training
1. Loss function
2. Optimizer
3. Training loop
4. Testing loop

In [131]:
# Define the loss function
lossFunction = nn.L1Loss()

# Define the optimizer
optimizer = torch.optim.SGD(linearModel.parameters(), lr=0.000001)


In [132]:
# Check the device of each data
print(f"x_train device: {x_train.device}")
print(f"x_test device: {x_test.device}")
print(f"y_train device: {y_train.device}")
print(f"y_test device: {y_test.device}")

x_train device: cpu
x_test device: cpu
y_train device: cpu
y_test device: cpu


In [133]:
# Set the data to the device --> Because we are using GPU for train the model

x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [134]:
# Train and Testing the model

torch.manual_seed(42)
epochs = 2000

# Track the loss values with each epoch
epoch_count = []
loss_values = []
test_loss_values = []

for epoch in range(epochs):
    ### Training phase
    # Set the model to train mode
    linearModel.train()

    # Forward porpagation/Forward pass (compute the output)
    yPred = linearModel(x_train)

    # Compute the loss
    loss = lossFunction(yPred, y_train)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward propagation
    loss.backward()

    # Update the parameters(Optimization of model parameters)
    optimizer.step()

    ### Testing phase
    # Set the model to evaluation mode
    linearModel.eval()

    with torch.inference_mode():
        # Forward porpagation
        yTestPred = linearModel(x_test)

        # Calculate the test loss
        test_loss = lossFunction(yTestPred, y_test)
        # print(linearModel.state_dict())

    # Track the loss values
    if epoch % 10 == 0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)

    # Print the loss values
        print(f"Epoch: {epoch}, Loss: {loss}, Test Loss: {test_loss}")

Epoch: 0, Loss: 2.8114452362060547, Test Loss: 6.0031023025512695
Epoch: 10, Loss: 2.798941135406494, Test Loss: 5.973101615905762
Epoch: 20, Loss: 2.786435842514038, Test Loss: 5.943100452423096
Epoch: 30, Loss: 2.7739312648773193, Test Loss: 5.913099765777588
Epoch: 40, Loss: 2.7614264488220215, Test Loss: 5.883098602294922
Epoch: 50, Loss: 2.7489216327667236, Test Loss: 5.853097915649414
Epoch: 60, Loss: 2.7364165782928467, Test Loss: 5.823096752166748
Epoch: 70, Loss: 2.723911762237549, Test Loss: 5.793097019195557
Epoch: 80, Loss: 2.71140718460083, Test Loss: 5.763095378875732
Epoch: 90, Loss: 2.6989026069641113, Test Loss: 5.733094215393066
Epoch: 100, Loss: 2.6863975524902344, Test Loss: 5.703093528747559
Epoch: 110, Loss: 2.6738929748535156, Test Loss: 5.673093318939209
Epoch: 120, Loss: 2.6613881587982178, Test Loss: 5.643091678619385
Epoch: 130, Loss: 2.648883581161499, Test Loss: 5.613091468811035
Epoch: 140, Loss: 2.636378526687622, Test Loss: 5.583090305328369
Epoch: 150, 

In [141]:
# Save the model
from pathlib import Path

# 1. Creaate models directory
MODEL_PATH = Path("Models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create the path to save the model
MODEL_NAME = "02_Linear Regression V2.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

# 3. Save the model state dict
torch.save(obj=linearModel.state_dict(), f=MODEL_SAVE_PATH)